In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from EDMtools import *
from SampTrajsTools import *
%matplotlib inline

In [ ]:
d=2 #dimension
M=5 #number of anchors
A = 10*np.random.rand(d,M)
#A = np.array([[1,5,8,2],[5,4,7,9]])
plt.scatter(A[0,:],A[1,:])

In [ ]:
N=50 #number of robot sample positions
K=4 #model complexity

C = 5*np.random.rand(d,K)
k=np.reshape(range(K),[K,1])
n=np.reshape(range(N),[1,N])
F = np.cos(np.pi*k*n/N)
R = C@F

#continuous trajectory for plotting
n_cont=np.reshape(np.linspace(0,N,1000),[1,1000])
F_cont = np.cos(np.pi*k*n_cont/N)
R_cont = C@F_cont

plt.scatter(A[0,:],A[1,:])
plt.scatter(R[0,:],R[1,:],color='orange')
plt.plot(R_cont[0,:],R_cont[1,:])
#plt.axis('off')
#plt.savefig('traj_setup.png')

In [ ]:
X = np.hstack([R,A])
G = X.transpose()@X
D = np.outer(np.ones(N+M),np.diag(G))+np.outer(np.diag(G),np.ones(N+M))-2*G

In [ ]:
D_tilde = D.copy()
D_tilde[:N,:N] = 0
DTR_tilde = D[:N,N:]

In [ ]:
print(checkStationaryPointSRLS(A,F,C,DTR_tilde))

In [ ]:
C_hat = customMDS(DTR_tilde, F, A)
print(C_hat)
print(C)

In [ ]:
R_hat = C_hat@F
R_hat_cont = C_hat@F_cont

plt.scatter(A[0,:],A[1,:])
plt.scatter(R[0,:],R[1,:])
plt.scatter(R_hat[0,:],R_hat[1,:])
plt.plot(R_cont[0,:],R_cont[1,:])
plt.plot(R_hat_cont[0,:],R_hat_cont[1,:])
plt.axis('off')

### Noisy case

In [ ]:
sigma = 40
DTR_tilde = D[:N,N:]+sigma*np.random.randn(N,M)
C_hat = customMDS(DTR_tilde+sigma*np.random.randn(N,M), F, A)
print(C_hat)
print(C)
print(checkStationaryPointSRLS(A,F,C_hat,DTR_tilde))
C_hat, costs = gradientDescent(A,F,C,DTR_tilde,maxIters=50)
print(checkStationaryPointSRLS(A,F,C_hat,DTR_tilde))
plt.plot(costs)

In [ ]:
R_hat = C_hat@F
R_hat_cont = C_hat@F_cont

plt.scatter(A[0,:],A[1,:])
plt.scatter(R[0,:],R[1,:])
plt.scatter(R_hat[0,:],R_hat[1,:])
plt.plot(R_cont[0,:],R_cont[1,:])
plt.plot(R_hat_cont[0,:],R_hat_cont[1,:])

### Missing measurements

In [ ]:
sigma = 2
DR = D[:,N:]
W = np.ones([M+N,M])
W[:N,:]*=np.random.rand(N,M)>0.9
print(W)
DR_tilde = W*DR
DR_tilde[:N,:]+=sigma*np.random.randn(N,M)
DR_hat, errs = alternateGDandKEonDR(DR_tilde, W, F, A, niter=400, DR_true=DR)
plt.plot(errs)
C_hat = customMDS(DR_hat[:N,:], F, A)
print(C_hat)
print(C)

In [ ]:
R_hat = C_hat@F
R_hat_cont = C_hat@F_cont

plt.scatter(A[0,:],A[1,:])
plt.scatter(R[0,:],R[1,:],color='orange')
plt.scatter(R_hat[0,:],R_hat[1,:],color='green')
plt.plot(R_cont[0,:],R_cont[1,:],color='orange')
plt.plot(R_hat_cont[0,:],R_hat_cont[1,:],color='green')
plt.axis('off')
#plt.savefig('traj_rec_with_noise.png')

In [ ]:
plt.scatter(A[0,:],A[1,:])
plt.plot(R_cont[0,:],R_cont[1,:])
R_useful = R[:,np.any(W[:N,:]!=0,axis=1)]
plt.scatter(R_useful[0,:],R_useful[1,:])
#plt.axis('off')
#plt.savefig('traj_useful.png')

In [ ]:
plt.scatter(A[0,:],A[1,:])
plt.plot(R_cont[0,:],R_cont[1,:])
for i in range(N):
    if np.sum(W[i,:])==1:
        plt.scatter(R[0,i],R[1,i],color='orange')
        
    if np.sum(W[i,:])==2:
        plt.scatter(R[0,i],R[1,i],color='red')

    if np.sum(W[i,:])>2:
        plt.scatter(R[0,i],R[1,i],color='green')
#plt.axis('off')
#plt.savefig('traj_useful_colorcoded.png')

In [ ]:
plt.plot(R_cont[0,:],R_cont[1,:])
plt.scatter(8,6)
plt.scatter(8,4,color='orange')
plt.scatter(8,2,color='red')
plt.scatter(8,0,color='green')
#plt.axis('off')
#plt.savefig('traj_key.png')

In [ ]:
DR_tilde = W*DR
DR_tilde[:N,:]+=sigma*np.random.randn(N,M)
DR_hat, errs = alternateGDandKEonDR(DR_tilde, W, F, A, niter=300, DR_true=DR)
plt.plot(errs)
C_hat = customMDS(DR_hat[:N,:], F, A)

In [ ]:
R_hat = C_hat@F
R_hat_cont = C_hat@F_cont

plt.scatter(A[0,:],A[1,:])
plt.scatter(R[0,:],R[1,:],color='orange')
plt.scatter(R_hat[0,:],R_hat[1,:],color='green')
plt.plot(R_cont[0,:],R_cont[1,:],color='orange')
plt.plot(R_hat_cont[0,:],R_hat_cont[1,:],color='green')
#plt.axis('off')
#plt.savefig('traj_rec.png')

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
tmp = DR[:N,:]
print(tmp[W[:N,:]==1])
print(W[:N,:]*tmp)

tmp = DR_tilde[:N,:]
print(tmp[W[:N,:]==1])
print(W[:N,:]*tmp)